In [ ]:
#Protwave --> capas CNN dilatadas

In [ ]:
# MusicVAE inspiration

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda
from tensorflow.keras.models import Model
import numpy as np

# --- 1. Definición de Parámetros ---
# Longitud de la secuencia de entrada (ej. 64 pasos de tiempo)
input_seq_length = 16
# Dimensión de cada elemento en la secuencia (ej. 128 características por paso)
features_dim = 128
# Dimensión del espacio latente
latent_dim = 64

# --- 2. Construcción del Codificador Jerárquico ---

# Capa de Entrada
# Define la forma de los datos de entrada: (batch_size, timesteps, features)
inputs = Input(shape=(input_seq_length, features_dim), name="entrada_secuencia")

# Pila de LSTMs (tu idea de resumen progresivo)
# Capa 1: Procesa la secuencia original.
# return_sequences=True es CRUCIAL para pasar la secuencia completa a la siguiente capa.
lstm_layer_1 = LSTM(units=128, return_sequences=True, name="lstm_resumen_1")(inputs)

# Capa 2: Recibe la secuencia de salidas de la capa 1 y la procesa de nuevo.
# Aprende patrones más abstractos a partir de los patrones de la capa 1.
lstm_layer_2 = LSTM(units=64, return_sequences=True, name="lstm_resumen_2")(lstm_layer_1)

# Capa 3: La última capa de la pila.
# return_sequences=False (valor por defecto) para que solo devuelva el *estado oculto final*.
# Este vector es el "resumen final" de toda la secuencia, procesado jerárquicamente.
final_summary_vector = LSTM(units=256, name="resumen_final_lstm")(lstm_layer_2)

# Capas Densas para generar el Espacio Latente
# Usamos el resumen final para predecir los parámetros de la distribución latente.
z_mean = Dense(latent_dim, name="z_mean")(final_summary_vector)
z_log_var = Dense(latent_dim, name="z_log_var")(final_summary_vector)

# --- 3. Muestreo del Espacio Latente (Sampling) ---
# Función para tomar la media y la varianza y generar un punto del espacio latente.
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    # Muestreo usando el "reparameterization trick"
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Capa Lambda para aplicar la función de muestreo
z = Lambda(sampling, output_shape=(latent_dim,), name="z_sampling")([z_mean, z_log_var])

# --- 4. Creación del Modelo Codificador Final ---
encoder = Model(inputs, [z_mean, z_log_var, z], name="codificador")

# Imprimimos un resumen de la arquitectura
encoder.summary()

In [ ]:
# Pooling entre capas LSTM para reducir la longitud de la secuencia

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, AveragePooling1D
from tensorflow.keras.models import Model

# Parámetros
input_seq_length = 16
features_dim = 128
latent_dim = 64

# Capa de Entrada
inputs = Input(shape=(input_seq_length, features_dim))

# --- Bloque 1: Resumen de 64 a 32 pasos ---
# La LSTM extrae características sin cambiar la longitud
lstm_out_1 = LSTM(128, return_sequences=True)(inputs)
# La capa de Pooling reduce la longitud de la secuencia a la mitad
pooled_out_1 = AveragePooling1D(pool_size=2)(lstm_out_1) # Shape de salida: (None, 32, 128)

# --- Bloque 2: Resumen de 32 a 16 pasos ---
lstm_out_2 = LSTM(64, return_sequences=True)(pooled_out_1)
pooled_out_2 = AveragePooling1D(pool_size=2)(lstm_out_2) # Shape de salida: (None, 16, 64)

# --- Capa final y espacio latente ---
# La última LSTM solo devuelve el resumen final (un vector)
final_summary_vector = LSTM(64)(pooled_out_2) # Shape de salida: (None, 64)

# Capas densas para el espacio latente
z_mean = Dense(latent_dim, name="z_mean")(final_summary_vector)
z_log_var = Dense(latent_dim, name="z_log_var")(final_summary_vector)

# ... (sigue la capa de muestreo Lambda, etc.)
# ...

# Modelo codificador
encoder = Model(inputs, [z_mean, z_log_var], name="encoder_con_pooling")
encoder.summary()

In [ ]:
# con capas encoder-decoder anidados
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, Lambda
from tensorflow.keras.models import Model

# --- Función reutilizable para el bloque de resumen ---
def create_summarizer_block(input_seq, output_seq_length, latent_dim, name_prefix):
    
    # Encoder
    _, state_h, state_c = LSTM(latent_dim, return_state=True, name=f"{name_prefix}_encoder_lstm")(input_seq)
    context_vector = [state_h, state_c]

    # Decoder
    decoder_input_seq = RepeatVector(output_seq_length, name=f"{name_prefix}_repeat_vector")(state_h)
    decoder_lstm = LSTM(latent_dim, return_sequences=True, name=f"{name_prefix}_decoder_lstm")

    # Conectar el contexto al decodificador
    output_seq = decoder_lstm(decoder_input_seq, initial_state=context_vector)
    return output_seq

# --- 1. Parámetros para el caso de uso de acordes ---
input_seq_length = 16   # 16 acordes en la secuencia
features_dim = 84      # 128 notas posibles (vector multi-hot por acorde)
final_latent_dim = 64   # Dimensión del espacio latente final

# --- 2. Construcción del Codificador VAE ---

# Entrada: (batch, 16, 84)
main_inputs = Input(shape=(input_seq_length, features_dim), name="progresion_acordes")

# --- Bloque 1: Resumen de 16 a 8 pasos ---
# Reducimos la secuencia de 16 a 8 acordes abstractos
summarized_seq_1 = create_summarizer_block(
    main_inputs, 
    output_seq_length=8, 
    latent_dim=128, # Dimensionalidad interna del bloque
    name_prefix="resumen_bloque_1"
)

# --- Capas Finales para el Espacio Latente ---
# Tomamos la secuencia final de 4 pasos y la resumimos en un único vector
final_summary_vector = LSTM(128, name="resumen_final_lstm")(summarized_seq_1)

# Proyectamos al espacio latente de 64 dimensiones
z_mean = Dense(final_latent_dim, name="z_mean")(final_summary_vector)
z_log_var = Dense(final_latent_dim, name="z_log_var")(final_summary_vector)

# Función de muestreo VAE
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, name="z_sampling")([z_mean, z_log_var])

# --- Creación del Modelo Codificador Final ---
chord_encoder = Model(main_inputs, [z_mean, z_log_var, z], name="Codificador_Acordes_Anidado")

print("### Resumen del Codificador para Progresiones de Acordes ###")
chord_encoder.summary(line_length=120)